In [368]:
--1
create view AllAccountRecords as
select  A.aid, A.pid, A.adate, A.abalance, A.aover, AR.rid, AR.rdate, AR.rtype, AR.ramount, AR.rbalance
from    accounts A
left outer JOIN accountrecords AR on A.aid = AR.aid;

Commands completed successfully

Total execution time: 00:00:00.008

In [369]:
--2
create function InsertBill()
returns trigger 
as $$ begin
    if (new.bamount < 0) then
        raise exception 'Negative amount on bill: %', new.bamount
        using errcode = '45000';
    elsif (new.bduedate < (TIMESTAMP 'tomorrow')) then 
        raise exception 'Bill due date too soon: %', new.bduedate
        using errcode = '45000';
    end if;
    return new;
end $$ language plpgsql;

create function Prevent()
returns trigger
as $$ begin
    raise exception 'Illegal Deletion/Update on table bills'
    using errcode = '45000';
end $$ language plpgsql;

create trigger PreventOperation
before update of bid, pid, bduedate, bamount
on bills 
for each row execute procedure Prevent();

create trigger ValidateBillInsertion
before insert
on bills 
for each row execute procedure InsertBill();

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.005

In [370]:
--3
create function InsertAccountRecord()
returns trigger 
as $$ 
declare 
    available integer;
    newbalance integer;
begin
    if (TG_OP = 'DELETE' or TG_OP = 'UPDATE') then
        raise exception 'Delete/Update forbidden on AccountRecords'
        using errcode = '45000';
    end if;

    -- retrieve existing information about account
    select  A.abalance+A.aover, A.abalance+new.ramount 
            into available, newbalance
    from    accounts A
    where   A.aid = new.aid;

    -- withdrawn amount can be minimum -available
    if (new.ramount < -available) then
        raise exception 'Invalid withdrawal on account: % \nAvailable: %\n Attempted withdrawal: %\n',new.aid,available,new.ramount
        using errcode = '45000';
    else 
        -- update balance on account
        update  accounts
        set     abalance = newbalance
        where   aid = new.aid;
        -- update balance on inserted record
        new.rbalance = newbalance;
        -- return new record
        return new;
    end if;
end $$ language plpgsql;

create trigger InsertNewAccountRecord 
before insert or update or delete
on accountrecords 
for each row execute procedure InsertAccountRecord();

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.010

In [371]:
--4
create function Transfer(
    iToAID integer,
    iFromAID integer,
    iAmount integer
) returns void 
as $$ begin
    if (iAmount < 0) then
        raise exception 'Cannot transfer a negative amount'
        using errcode = '45000';
    end if;

    insert into accountrecords(aid,rdate,rtype,ramount)        
    values  (IToAID, current_date, 'T', iAmount),
            (IFromAID, current_date, 'T', -iAmount);
end $$ language plpgsql;

Commands completed successfully

Total execution time: 00:00:00.001

In [372]:
--5
create view DebtorStatus as 
select  P.pid, P.pname, sum(A.abalance) as totalbalance, (select -sum(AA.abalance) from accounts AA where AA.pid = P.pid and AA.abalance < 0) as totaloverdraft
from    people P
join    accounts A on A.pid = P.pid
GROUP BY P.pid
HAVING  sum(A.abalance) < 0;

Commands completed successfully

Total execution time: 00:00:00.004

In [373]:
--6
create function AutoCreateAccount() returns trigger 
as $$ begin
    insert into Accounts(pid,adate,abalance,aover)
    values (new.pid, CURRENT_DATE, 0, 10000);
    return new;
end $$ language plpgsql;

create trigger CreateAccountOnNewPerson
after insert on people
for each row EXECUTE procedure AutoCreateAccount();

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.017

In [374]:
--7
create function InsertPerson (
    IN iName varchar,
    IN iGender char(1),
    IN iHeight float, 
    IN iAmount integer
) returns void 
as $$ 
declare 
    newacc integer;
begin
    insert into people (pname,pgender,pheight)
    values (iName, iGender, iHeight);    
    
    select lastval() into newacc;

    insert into accountrecords(aid,rdate,rtype,ramount)
    values (newacc, CURRENT_DATE ,'T', iAmount);
end $$ language plpgsql;

Commands completed successfully

Total execution time: 00:00:00.001

In [375]:
--8
create function PayOneBill (
    IN iBID integer
) returns void 
as $$ 
declare
    id integer;
    ispaid boolean;
    amount integer;
    accid integer;
begin
    select  pid, bamount, bispaid into id, amount, ispaid
    from    bills B
    where   B.bid = iBID;

    select  max(A.aid) into accid
    from    accounts A 
    where   A.pid = id and (A.aover+A.abalance) = (
        select  max(aover+abalance)
        from    accounts
        where   pid = id
    );

    if (ispaid) then
        raise exception 'Bill already paid: %',iBID
        using errcode = '45000';
    elsif (id is null) then
        raise exception 'Bill does not exist: %',iBID
        using errcode = '45000';
    elsif (accid is null) then 
        raise exception 'No account available for personid: %', id
        using errcode = '45000';
    end if;

    insert into accountrecords(aid, rdate, rtype, ramount)
    values (
        accid,
        CURRENT_DATE,
        'B',
        - amount
    );

    update  bills
    set     bispaid = TRUE
    where   bid = iBID;
end $$ language plpgsql;

Commands completed successfully

Total execution time: 00:00:00.002

In [376]:
--9
create function LoanMoney (
    IN iAID integer,
    IN iAmount integer, 
    IN iDueDate Date
) returns void 
as $$ 
declare 
    pid integer;
begin 
    
    insert into accountrecords(aid, rdate, rtype, ramount)
    values (iAID, CURRENT_DATE, 'L', iAmount);

    select  A.pid into pid
    from    accounts A 
    where   A.aid = iAID;

    insert into bills(pid, bduedate, bamount, bispaid)
    values (pid, iDueDate, iAmount, FALSE);

end $$ language plpgsql;

Commands completed successfully

Total execution time: 00:00:00.001

In [377]:
--10
create view FinancialStatus as 
select  P.pid, P.pname, 
        case when (sum(A.abalance) is null) then 0
        else sum(A.abalance) end as balance, 
        case when (sum(B.bamount) is null) then 0
        else sum(B.bamount) end as unpaid
from    accounts A
JOIN    people P on P.pid = A.pid
left outer JOIN    bills B on P.pid = B.pid and B.bispaid = FALSE
group by P.pid, P.pname;

Commands completed successfully

Total execution time: 00:00:00.004